In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from modules.CARLA import carla
from modules.CFEC.cfec.explainers import Fimap, Cadex
from models.tfmodel import TFModelAdult

import pandas as pd 
import numpy as np




Appending path to carla module D:\Research\ecemosp\src\modules\CARLA


c:\Users\ignacy\.conda\envs\cfec\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\ignacy\.conda\envs\cfec\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\Users\ignacy\.conda\envs\cfec\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


Appending path to carla module D:\Research\ecemosp\src\modules\CARLA
[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
from carla.data.catalog import CsvCatalog
import json

with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f)

continuous = constraints['continuous']
categorical = constraints['categorical']
immutable = constraints['immutable']
columns_order = constraints['features_order']

dataset = CsvCatalog(file_path="../data/adult_prep.csv",
                    continuous=continuous,
                    categorical=categorical,
                    immutables=immutable,
                    target='income')
                    
dataset.df.head(2)

,age,education.num,capital.gain,capital.loss,hours.per.week,...,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,1.000000,0.533333,0.0,1.0,0.397959,...,0.0,0.0,0.0,0.0,0.0
1,0.890411,0.533333,0.0,1.0,0.173469,...,1.0,0.0,0.0,0.0,0.0


In [3]:
model = TFModelAdult(dataset)

model_predictions = model.predict(dataset.df_train)
model_predictions = np.argmax(model_predictions, axis=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                5824      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 2)                 3

c:\Users\ignacy\.conda\envs\cfec\lib\site-packages\keras\engine\training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


FIMAP


In [4]:
fimap = Fimap()

# Sample random 5000 records
sample = np.arange(0, dataset.df_train.shape[0])
np.random.shuffle(sample)
sample = sample[:50]



#fimap.fit(dataset.df_train[columns_order].iloc[sample], model_predictions[sample])
# cf_fimap = fimap.generate(dataset.df_test[columns_order].iloc[0])
# cf_fimap

CADEX

In [5]:
factual = dataset.df_test[columns_order].iloc[0]

cadex = Cadex(model, n_changed=5)
cf = cadex.generate(factual)
cf

<tf.Variable 'Variable:0' shape=(1, 90) dtype=float64>
<tf.Variable 'Variable:0' shape=(1, 90) dtype=float64>


AttributeError: 'Tensor' object has no attribute 'numpy'

In [ ]:
import tensorflow as tf 
from tensorflow.keras.losses import CategoricalCrossentropy

def _get_gradient(x, y_true):
    with tf.GradientTape() as t:
        t.watch(x)
        y_pred = model(x)
        _loss = CategoricalCrossentropy()
        loss = _loss([y_true], y_pred)

        print(loss)
        
        grad = t.gradient(loss, x)

    print(grad)

x = dataset.df_test[columns_order].iloc[0]
y_true = [0, 1]

_get_gradient(tf.Variable(x.to_numpy().reshape((1,90))), y_true)


In [ ]:
np.array([1,2]).tolist()

In [ ]:
# import tensorflow as tf 

# # To numpy
# s = tf.Variable([1,2,3])
# with tf.compat.v1.Session() as sess:
#     sess.run(tf.compat.v1.global_variables_initializer())
#     z = s.eval(session=sess)
# z